# Quasi static forces from model test

# Purpose
Calculating the quasi static forces from a model test by assuming the added masses. This can then be compared to the static VCT calculations.

# Setup

In [ ]:
# %load imports.py
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl
#import vessel_manoeuvring_models.models.nonlinear_martin_vmm as vmm
#import vessel_manoeuvring_models.nonlinear_martin_vmm_equations as eq

#import vessel_manoeuvring_models.models.linear_vmm as vmm
#import vessel_manoeuvring_models.nonlinear_vmm_equations as eq
import vessel_manoeuvring_models.nonlinear_martin_vmm_equations as eq

#import vessel_manoeuvring_models.models.linear_vmm as model
from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.plot import track_plot
from vessel_manoeuvring_models.equation import Equation

# Methodology
Generally:

$ F = m \cdot a $

if $m$ (including added mass) can is known $F$ should be possible to calculate from a dynamic model test.

For the ship in the Y-direction this can be written as:

## Y:

In [ ]:
Math(vlatex(eq.Y_eom))

In [ ]:
Math(vlatex(eq.fy_eq))

In [ ]:
Y_eq = eq.Y_eom.subs(eq.Y_nonlin,sp.solve(eq.fy_eq,Y_nonlin)[0])
Math(vlatex(Y_eq))

In [ ]:
eq_Y_qs = sp.Eq(Y_qs,
      sp.solve(Y_eq, Y_qs)[0])
Math(vlatex(eq_Y_qs))

In [ ]:
subs = {value:key for key,value in eq.p.items()}
Y_qs_lambda = lambdify(eq_Y_qs.subs(subs).rhs)

## X:

In [ ]:
Math(vlatex(eq.X_eom))

In [ ]:
Math(vlatex(eq.fx_eq))

In [ ]:
X_eq = eq.X_eom.subs(eq.X_nonlin,sp.solve(eq.fx_eq,X_nonlin)[0])
Math(vlatex(X_eq))

In [ ]:
eq_X_qs = sp.Eq(X_qs,
      sp.solve(X_eq, X_qs)[0])
Math(vlatex(eq_X_qs))

In [ ]:
subs = {value:key for key,value in eq.p.items()}
X_qs_lambda = lambdify(eq_X_qs.subs(subs).rhs)

## N:

In [ ]:
Math(vlatex(eq.N_eom))

In [ ]:
Math(vlatex(eq.mz_eq))

In [ ]:
N_eq = eq.N_eom.subs(eq.N_nonlin,sp.solve(eq.mz_eq,N_nonlin)[0])
Math(vlatex(N_eq))

In [ ]:
eq_N_qs = sp.Eq(N_qs,
      sp.solve(N_eq, N_qs)[0])
Math(vlatex(eq_N_qs))

In [ ]:
N_qs_lambda = lambdify(eq_N_qs.subs(subs).rhs)

In [ ]:
u1d,v1d,r1d = sp.symbols('u1d, v1d, r1d')
subs = [
    (u1d,u1d),
    (v1d,v1d),
    (r1d,r1d),

]
eq_X_ = X_eq.subs(subs)
eq_Y_ = Y_eq.subs(subs)
eq_N_ = N_eq.subs(subs)


A,b = sp.linear_eq_to_matrix([eq_X_,eq_Y_,eq_N_],[u1d,v1d,r1d])
#A,b = sp.linear_eq_to_matrix([eq_Y_,eq_N_],[v1d,r1d])

In [ ]:
Math(vlatex(eq_Y_))

In [ ]:
A

In [ ]:
Math(vlatex(b))

In [ ]:
lambdify(A.inv()*b)

# Quasi static forces from model tests

## Load test

In [ ]:
df_runs = mdl.runs()

In [ ]:
#id=22773
#id=22616
id=22774
#id=22770


df, units, meta_data = mdl.load(id=id, dir_path='../data/processed/kalman')
df.index = df.index.total_seconds()
df = df.iloc[0:-100].copy()
df.index-=df.index[0]
df.sort_index(inplace=True)
df['-delta'] = -df['delta']
df['V'] = np.sqrt(df['u']**2 + df['v']**2)

In [ ]:
meta_data['rho']=1000
meta_data['mass'] = meta_data['Volume']*meta_data['rho']
meta_data.dropna()

In [ ]:
df.head()

In [ ]:
df['thrust'] = df['Prop/PS/Thrust'] + df['Prop/SB/Thrust']
df['U'] = df['V']

In [ ]:
from vessel_manoeuvring_models.visualization.plot import track_plot
fig,ax=plt.subplots()
#fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

# Ship parameters

In [ ]:
T_ = (meta_data.TA + meta_data.TF)/2
L_ = meta_data.lpp
m_ = meta_data.mass
rho_ = meta_data.rho
B_ = meta_data.beam
CB_ = m_/(T_*B_*L_*rho_)
I_z_ = m_*meta_data.KZZ**2
#I_z_ = 900


ship_parameters = {
        'T' : T_,
        'L' : L_,
        'CB' :CB_,
        'B' : B_,
        'rho' : rho_,
        'x_G' : 0,  # motions are expressed at CG
        'm' : m_,
        'I_z': I_z_, 
        'volume':meta_data.Volume,
    }

ps = prime_system.PrimeSystem(**ship_parameters)  # model

scale_factor = meta_data.scale_factor
ps_ship = prime_system.PrimeSystem(L=ship_parameters['L']*scale_factor, rho=meta_data['rho'])  # ship

In [ ]:
ship_parameters

# Brix parameters

In [ ]:
def calculate_prime(row, ship_parameters):
    return run(function=row['brix_lambda'], inputs=ship_parameters)


mask = df_parameters['brix_lambda'].notnull()
df_parameters.loc[mask,'brix_prime'] = df_parameters.loc[mask].apply(calculate_prime, ship_parameters=ship_parameters, axis=1)

df_parameters['brix_prime'].fillna(0, inplace=True)
#df_parameters['brix_SI'].fillna(0, inplace=True)



In [ ]:
df_parameters.head()

In [ ]:
df_parameters['prime'] = df_parameters['brix_prime']

def to_SI(row):
    
    prime = row['prime']
    denominator = row['denominator']
    lamda = lambdify(denominator)
    try:
        denominator_value = run(lamda, inputs=ship_parameters)
        SI = prime*denominator_value
    except:
        return None
    else:
        return SI
    
df_parameters['brix_SI'] = df_parameters.apply(to_SI, axis=1)
    

In [ ]:
df['Y_qs'] = run(Y_qs_lambda, inputs=ship_parameters, **df, **df_parameters['brix_SI'])
df['X_qs'] = run(X_qs_lambda, inputs=ship_parameters, **df, **df_parameters['brix_SI'])
df['N_qs'] = run(N_qs_lambda, inputs=ship_parameters, **df, **df_parameters['brix_SI'])

In [ ]:
keys = ['X_qs','Y_qs','N_qs','delta']
for key in keys:
    fig,ax=plt.subplots()
    fig.set_size_inches(15,2.5)
    df.plot(y=key, ax=ax)
    ax.grid(True)

In [ ]:
sns.pairplot(df, x_vars=['u','v','r','delta'], y_vars=['X_qs','Y_qs','N_qs'])

# Regression

## N

In [ ]:
Math(vlatex(eq.N_qs_eq))

In [ ]:
N_ = sp.symbols('N_')
diff_eq_N = regression.DiffEqToMatrix(ode=eq.N_qs_eq.subs(N_qs,N_), 
                                      label=N_, base_features=[delta,u,v,r])

In [ ]:
diff_eq_N.eq_beta

In [ ]:
Math(vlatex(diff_eq_N.eq_X))

In [ ]:
diff_eq_N.eq_y

In [ ]:
diff_eq_N.y_lambda

In [ ]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
def show_pred(X,y,results, label):
    
    display(results.summary())
    
    X_ = X
    y_ = y
    y_pred = results.predict(X_)
    
    prstd, iv_l, iv_u = wls_prediction_std(results, exog=X_, alpha=0.05)
    #iv_l*=-1 
    #iv_u*=-1
    
    fig,ax=plt.subplots()
    ax.plot(X_.index,y_, label='Numerical gradient from model test')
    ax.plot(X_.index,y_pred, '--', label='OLS')
    ax.set_ylabel(label)
    
    ax.fill_between(X_.index, y1=iv_l, y2=iv_u, zorder=-10, color='grey', alpha=0.5, label=r'5\% confidence')
    ax.legend();

In [ ]:
X = diff_eq_N.calculate_features(data=df)
y = diff_eq_N.calculate_label(y=df['N_qs'])

model_N = sm.OLS(y,X)
results_N = model_N.fit()

show_pred(X=X,y=y,results=results_N, label=r'$N_{qs}$')

## Y

In [ ]:
Y_ = sp.symbols('Y_')
diff_eq_Y = regression.DiffEqToMatrix(ode=eq.Y_qs_eq.subs(Y_qs,Y_), 
                                      label=Y_, base_features=[delta,u,v,r])

In [ ]:
X = diff_eq_Y.calculate_features(data=df)
y = diff_eq_Y.calculate_label(y=df['Y_qs'])

model_Y = sm.OLS(y,X)
results_Y = model_Y.fit()

show_pred(X=X,y=y,results=results_Y, label=r'$Y_{qs}$')

## X

In [ ]:
X_ = sp.symbols('X_')
diff_eq_X = regression.DiffEqToMatrix(ode=eq.X_qs_eq.subs(X_qs,X_), 
                                      label=X_, base_features=[delta,u,v,r,thrust])

In [ ]:
X = diff_eq_X.calculate_features(data=df)
y = diff_eq_X.calculate_label(y=df['X_qs'])

model_X = sm.OLS(y,X)
results_X = model_X.fit()

show_pred(X=X,y=y,results=results_X, label=r'$X_{qs}$')

In [ ]:
results_summary_X = regression.results_summary_to_dataframe(results_X)
results_summary_Y = regression.results_summary_to_dataframe(results_Y)
results_summary_N = regression.results_summary_to_dataframe(results_N)

## Add the regressed parameters
Hydrodynamic derivatives that depend on acceleration cannot be obtained from the VCT regression. They are however essential if a time simulation should be conducted. These values have then been taken from Brix semi empirical formulas for the simulations below.

In [ ]:
df_parameters_all = df_parameters.copy()
for other in [results_summary_X, results_summary_Y, results_summary_N]:
    df_parameters_all = df_parameters_all.combine_first(other)

df_parameters_all.rename(columns={'coeff':'regressed'}, inplace=True)
df_parameters_all.drop(columns=['brix_lambda'], inplace=True)

df_parameters_all['SI'] = df_parameters_all['regressed'].combine_first(df_parameters_all['brix_SI'])  # prefer regressed
df_parameters_all['SI'].fillna(0,inplace=True)

In [ ]:
df_parameters_all.head()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(15,5)
mask = ((df_parameters_all['brix_prime'] != 0) |
        (df_parameters_all['regressed']).notnull())
        
df_parameters_plot = df_parameters_all.loc[mask]
df_parameters_plot.plot.bar(y=['brix_SI','regressed'], ax=ax);

# Simulate captive

In [ ]:
subs = {value:key for key,value in eq.p.items()}
X_qs_lambda = lambdify(eq.X_qs_eq.rhs.subs(subs))
Y_qs_lambda = lambdify(eq.Y_qs_eq.rhs.subs(subs))
N_qs_lambda = lambdify(eq.N_qs_eq.rhs.subs(subs))

In [ ]:
df['X_qs_pred'] = run(function=X_qs_lambda, inputs=df, **df_parameters_all['SI'])
df['Y_qs_pred'] = run(function=Y_qs_lambda, inputs=df, **df_parameters_all['SI'])
df['N_qs_pred'] = run(function=N_qs_lambda, inputs=df, **df_parameters_all['SI'])


In [ ]:
for key in ['X_qs','Y_qs','N_qs']:
    fig,ax=plt.subplots()
    df.plot(x=key, y=f'{key}_pred',style='.', ax=ax)
    ax.plot([df[key].min(), df[key].max()], [df[key].min(), df[key].max()], 'r-')

# Simulation

In [ ]:
#X_eq = eq.X_eq.copy()
#Y_eq = eq.Y_eq.copy()
#N_eq = eq.N_eq.copy()
#
#eqs = [X_eq, Y_eq, N_eq]
#solution = sp.solve(eqs, u1d, v1d, r1d, dict=True)
#
### Decouple the equations:
#u1d_eq = sp.Eq(u1d, solution[0][u1d]) 
#v1d_eq = sp.Eq(v1d, solution[0][v1d]) 
#r1d_eq = sp.Eq(r1d, solution[0][r1d])
#
#
### Lambdify:
#subs = {value:key for key,value in eq.p.items()}
#u1d_lambda = lambdify(u1d_eq.subs(subs).rhs)
#v1d_lambda = lambdify(v1d_eq.subs(subs).rhs)
#r1d_lambda = lambdify(r1d_eq.subs(subs).rhs)


u1d,v1d,r1d = sp.symbols('u1d, v1d, r1d')
subs = [
    (u1d,u1d),
    (v1d,v1d),
    (r1d,r1d),

]
eq_X_ = X_eq.subs(subs)
eq_Y_ = Y_eq.subs(subs)
eq_N_ = N_eq.subs(subs)


A,b = sp.linear_eq_to_matrix([eq_X_,eq_Y_,eq_N_],[u1d,v1d,r1d])

acceleartion_eq = A.inv()*b
## Lambdify:
subs = {value:key for key,value in eq.p.items()}
subs[X_qs] = sp.symbols('X_qs')
subs[Y_qs] = sp.symbols('Y_qs')
subs[N_qs] = sp.symbols('N_qs')

acceleartion_lambda = lambdify(acceleartion_eq.subs(subs))

In [ ]:
A

In [ ]:
b

In [ ]:
from scipy.spatial.transform import Rotation as R

def step(t, states, parameters, ship_parameters, control):

    u,v,r,x0,y0,psi = states
    
    #if u < 0:
    #    dstates = [
    #    0,
    #    0,
    #    0,
    #    0,
    #    0,
    #    0,
    #    ]    
    #    return dstates
    
    states_dict = {
        'u':u,
        'v':v,
        'r':r,
        'x0':x0,
        'y0':y0,
        'psi':psi,
        
        }
    
    inputs = dict(parameters)
    inputs.update(ship_parameters)
    inputs.update(states_dict)
    
    if isinstance(control, pd.DataFrame):
        index = np.argmin(np.array(np.abs(control.index - t)))
        control_ = dict(control.iloc[index])
    else:
        control_ = control
    inputs.update(control_)
    
    #inputs['U'] = np.sqrt(u**2 + v**2)  #Instantanious velocity
    
    inputs['X_qs'] = run(function=X_qs_lambda, inputs=inputs)
    inputs['Y_qs'] = run(function=Y_qs_lambda, inputs=inputs)
    inputs['N_qs'] = run(function=N_qs_lambda, inputs=inputs)
    
    u1d,v1d,r1d = run(function=acceleartion_lambda, inputs=inputs)
    
    rotation = R.from_euler('z', psi, degrees=False)
    w = 0
    velocities = rotation.apply([u,v,w])
    x01d = velocities[0]
    y01d = velocities[1]
    psi1d = r    
    dstates = [
        u1d,
        v1d,
        r1d,
        x01d,
        y01d,
        psi1d,
    ]    
    return dstates

In [ ]:
def simulate(df_, parameters, ship_parameters):
    
    t = df_.index
    t_span = [t.min(),t.max()]
    t_eval = np.linspace(t.min(),t.max(),len(t))
    
    control = df_[['delta','thrust']]
    #control = {
    #    'delta': float(df_['delta'].mean()),
    #    'thrust': float(df_['thrust'].mean()),
    #    
    #}
    
    df_0 = df_.iloc[0:100].median(axis=0)
    y0 = {
        'u' : df_0['u'], 
        'v' : df_0['v'],
        'r' : df_0['r'],
        'x0' : df_0['x0'],
        'y0' : df_0['y0'],
        'psi' : df_0['psi']
        }
    
    solution = solve_ivp(fun=step, t_span=t_span, y0=list(y0.values()), t_eval=t_eval, 
                args=(parameters, ship_parameters, control))
    
    columns = list(y0.keys())
    df_result = pd.DataFrame(data=solution.y.T, columns=columns)
    df_result.index=t[0:len(df_result)]
    
    df_result['beta'] = -np.arctan2(df_result['v'],df_result['u'])
    
    return solution, df_result

In [ ]:
df_cut = df.iloc[2000:].copy()
#df_cut = df.copy()
df_cut.index-=df_cut.index[0]
solution, df_result = simulate(df_cut, parameters = df_parameters_all['SI'], 
                               ship_parameters=ship_parameters)

In [ ]:
fig,ax=plt.subplots()
track_plot(df=df_cut, lpp=ship_parameters['L'], beam=ship_parameters['B'],ax=ax, label='model test')
track_plot(df=df_result, lpp=ship_parameters['L'], beam=ship_parameters['B'],ax=ax, label='simulation', color='green')
ax.legend()


for key in df_result:
    fig,ax = plt.subplots()
    df_cut.plot(y=key, label='model test', ax=ax)
    df_result.plot(y=key, label='simulation', ax=ax)
    ax.set_ylabel(key)